In [27]:
import pandas as pd
import tqdm
from rouge import Rouge 

In [2]:
from models.unsupervised import kMeans

In [3]:
from UniversalSentenceEncoder import USEEmbedder
from Preprocessors import PlaceboPreprocessor

In [4]:
test_data = pd.read_pickle("./training_data/test_raw.pkl")

In [5]:
test_data = test_data.sort_values(by=['Language'])

In [6]:
test_data.head()

,index,Lead,Body,Language,ID,isTrain
3,3,"Rod Eddington, BA's chief executive, said the ...",High fuel prices hit BA's profits British Air...,English,3,False
1465,1465,Chester have won only two of their last 10 Lea...,Rush future at Chester uncertain Ian Rush's f...,English,1465,False
1463,1463,Victory took Real to within six points of lead...,Owen delighted with Real display Michael Owen...,English,1463,False
1456,1456,"""We got our goals early and in the minds of so...",Souness delight at Euro progress Boss Graeme ...,English,1456,False
1454,1454,"""For a fourth-choice striker at the start I'm ...",Owen determined to stay in Madrid England for...,English,1454,False


In [7]:
summarizer = kMeans(USEEmbedder, PlaceboPreprocessor)

In [50]:
summaries = []

In [51]:
flatdict = {}
rouge = Rouge()

In [52]:
for i, row in tqdm.tqdm_notebook(test_data.iterrows(), total=len(test_data.index)):
    try:
        smry = summarizer.summarize(row.Body, row.Language, 0.3, sif=True)
    except:
        smry = " "
    summaries.append(smry)
    flatlist = []
    scores = rouge.get_scores(smry, row.Lead)[0]
    for metric in scores:
        for key in scores[metric]:
            flatlist.append(scores[metric][key])
    flatdict[i] = flatlist

/home/swrdata/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))
/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))
/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))
/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))
/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))
/home/swrdata/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_k_means.py:88: RuntimeWarn

In [58]:
test_data["Summary_USE"] = summaries

In [59]:
r_scores = pd.DataFrame.from_dict(flatdict, orient="index",
                       columns=['R1_f', 'R1_p', 'R1_r', 'R2_f', 'R2_p', 'R2_r','Rl_f', 'Rl_p', 'Rl_r'])

In [61]:
test_data = pd.merge(test_data, r_scores, left_index=True, right_index=True)

In [62]:
test_data.head()

,index,Lead,Body,Language,ID,isTrain,Summary_USE,R1_f,R1_p,R1_r,R2_f,R2_p,R2_r,Rl_f,Rl_p,Rl_r
3,3,"Rod Eddington, BA's chief executive, said the ...",High fuel prices hit BA's profits British Air...,English,3,False,High fuel prices hit BA's profits British Air...,0.392962,0.496296,0.325243,0.188791,0.238806,0.156098,0.366667,0.419048,0.325926
1465,1465,Chester have won only two of their last 10 Lea...,Rush future at Chester uncertain Ian Rush's f...,English,1465,False,"Vaughan said: ""I offered a financial settlemen...",0.494845,0.657534,0.396694,0.406250,0.541667,0.325000,0.523490,0.639344,0.443182
1463,1463,Victory took Real to within six points of lead...,Owen delighted with Real display Michael Owen...,English,1463,False,"""I felt good all though the game and it is obv...",0.472727,0.609375,0.386139,0.368098,0.476190,0.300000,0.511278,0.576271,0.459459
1456,1456,"""We got our goals early and in the minds of so...",Souness delight at Euro progress Boss Graeme ...,English,1456,False,An early own goal followed by an Alan Shearer ...,0.473988,0.694915,0.359649,0.385965,0.568966,0.292035,0.468750,0.612245,0.379747
1454,1454,"""For a fourth-choice striker at the start I'm ...",Owen determined to stay in Madrid England for...,English,1454,False,Owen determined to stay in Madrid England for...,0.527397,0.712963,0.418478,0.427586,0.579439,0.338798,0.543590,0.662500,0.460870


In [64]:
test_data.R2_f.describe()

count    6315.000000
mean        0.074994
std         0.124474
min         0.000000
25%         0.000000
50%         0.028070
75%         0.074227
max         0.809524
Name: R2_f, dtype: float64